# 支持连续对话的GPT4

In [ ]:
import requests
from IPython.display import display, Markdown
import ipywidgets as widgets

def query_bing_api(message):
    url = "http://api.starchent.top/API/bing.php"
    params = {
        "msg": message,
        "modle": "down",
        "type": "text"
    }

    response = requests.post(url, params=params)
    data = response.text

    if response.status_code == 200:
        return data
    else:
        return "抱歉，无法获取响应内容。"

text_input = widgets.Textarea(
    value='',
    placeholder='请输入问题',
    description='',
    layout=widgets.Layout(width='auto', height='200px')
)
button = widgets.Button(description='提交')
restart_button = widgets.Button(description='重新开始')

output = widgets.Output(layout={
    'border': '1px solid black'
})

history = []

def on_button_clicked(b):
    user_input = text_input.value

    full_input = '||'.join(history + [user_input])
    full_input = full_input.replace('"', '单引号')
    full_input = full_input.replace("'", '双引号')
    full_input = full_input.replace("\n", '换行')

    response = query_bing_api(full_input)

    with output:
        display(Markdown(f"**用户：** {user_input}"))
        display(Markdown(f"**回答：** {response}"))

        history.append(user_input)
        history.append(response)

        text_input.value = ''
        display(Markdown("----------"))  # 在每一轮对话后添加分隔符

def on_restart_button_clicked(b):
    history.clear()
    output.clear_output()
    text_input.value = ''

button.on_click(on_button_clicked)
restart_button.on_click(on_restart_button_clicked)

layout = widgets.VBox([text_input, widgets.HBox([button, restart_button]), output])

display(layout)


# MidJourney Ai绘画

In [ ]:
import requests
import json
from PIL import Image
from io import BytesIO
import time

# 创建请求头
headers = {
    "UUID": "afd22831-6e9e-c4e4-2477-825a3328a178"
}

def generate_and_show_image(prompt):
    # 设置请求的URL
    url = "http://midjourney-api.ai-des.com/func2api/Imagine"

    # 创建要发送的数据
    data = {
        "type": "P",
        "prompt": prompt,
        "imgId": "",
        "num": 0
    }

    # 使用requests发送POST请求
    response = requests.post(url, json=data, headers=headers)

    img_id = None
    if response.status_code == 200:
        # 如果响应状态为200(成功)，则解析响应内容
        response_data = response.json()
        img_id = response_data['data']['imgId']
        print(f"Image ID: {img_id}")
    else:
        print(f"请求失败，状态码：{response.status_code}")

    def get_image_url(img_id):
        url = "http://midjourney-api.ai-des.com/func2api/GetPicByImgID"

        data = {
            "imgId": img_id
        }

        while True:
            response = requests.post(url, json=data, headers=headers)
            if response.status_code == 200:
                response_data = response.json()
                img_url = response_data['data']['mjSrcUrl']
                if img_url:  # 图片链接已生成
                    return img_url
            else:
                print(f"请求失败，状态码：{response.status_code}")
            time.sleep(2)  # 每2秒检查一次

    if img_id:
        img_url = get_image_url(img_id)
        print(f"Image URL: {img_url}")

        # 请求图片
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))

        # 展示图片
        img.show()

# 使用函数生成并显示图片
generate_and_show_image("千山鸟飞绝，万径人踪灭")
